#  09. Продвинутые темы 1. <br> Исключения. Тестирование, логирование. <br> Файлы. Модульное оформление. Argparse.  

---

##  Исключения   [DOCS](https://docs.python.org/3/library/exceptions.html)

Давайте разберем важный тип объектов в Питоне, без которого крупная (да и не очень) программа не обойдется. 

Что такое исключения (т.е. исключительные ситуации) или exceptions? Например, к чему приведет попытка чтения несуществующего файла? Или если файл был случайно удален пока программа работала? Когда в программе происходит какая-либо ошибка, будь это просто пропущенная запятая/скобка, деление на ноль или неверные аргументы функции, вызывается **исключение**, которое останавливает ход программы (обычно) и печатает в вывод ошибок ([stderr](https://docs.python.org/3/library/sys.html#sys.stderr)) описание возникшей проблемы. 
То есть исключения просто сообщают программисту об ошибках, после чего он может либо исправить баг (bug), либо обработать исключение верным образом. 

Например:

In [7]:
print(2 1)

SyntaxError: invalid syntax (<ipython-input-7-791f33c93b8d>, line 1)

In [113]:
0 / 0

ZeroDivisionError: division by zero

Исключения имеют несколько классов, часть из них приведена ниже (подробное описание можно почитать в [документации](https://docs.python.org/3/library/exceptions.html#concrete-exceptions) или [тут](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html)), некоторые вы могли уже не раз встретить.

```
BaseException
 +-- SystemExit
 +-- KeyboardInterrupt
 +-- Exception
      +-- StopIteration
      +-- ArithmeticError
      |    +-- FloatingPointError
      |    +-- OverflowError
      |    +-- ZeroDivisionError
```

```
      +-- AssertionError
      +-- AttributeError
      +-- EOFError
      +-- ImportError
      |    +-- ModuleNotFoundError
      +-- LookupError
      |    +-- IndexError
      |    +-- KeyError
```

```
      +-- MemoryError
      +-- NameError
      |    +-- UnboundLocalError
      +-- RuntimeError
      |    +-- NotImplementedError
      |    +-- RecursionError
```

```
      +-- SyntaxError
      |    +-- IndentationError
      |         +-- TabError
      +-- SystemError
      +-- TypeError
      +-- ValueError
      +-- Warning
```

In [54]:
import sys
import os
import math

**Traceback** (трейс) - подсказки и имя ошибки, которые дают нам полную информацию о том, где порождено исключение (указывает номер строки и иногда место в строке), и с чем оно связано (после имени ошибки обычно идет более подробное описание, что не так). 

Далее попробуйте раскомментировать по очереди каждый из примеров (удобнее это делать с помощью одновременного нажатия Ctrl и /).

In [59]:
#---------------------
# sys.exit()
#---------------------
# print(math.exp(709.7828))
#---------------------
# 7 / 0
#---------------------
assert 10 != 10
#--------------------- 

AssertionError: 

In [64]:
#---------------------
# math.func()
#---------------------
# import module
#---------------------
# a = [1]; a[1]
#---------------------
# d = {}; d['key']
#---------------------

In [68]:
#---------------------
# b
#---------------------
# def f(): c += 1 
# f()
#---------------------
# def f(): return f()
# f()
#---------------------

In [91]:
#---------------------
# print 1
#---------------------
# abs(1
#---------------------
# 1a
#---------------------
#     1
#---------------------
# def f():
# return 1
#---------------------
# abs('ten')
#---------------------
# float('ten')
#--------------------- 
os.sys.exit()

1


SystemExit: 

/home/svdcvt/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


###  Вызов исключений - `raise`  

Возникают ситуации, когда исключения необходимо "бросать", "вызывать", "рейзить", т.е. написать код, который при исполнении будет инициировать исключительную ситуацию.

Например, функция, которая решает квадратное уравнение. Вы условились, что корни только вещественные, тогда в случае комплексных корней стоит бросить исключение. Или в вашей функции необходимо проверить в самом начале, что аргументы нужного типа (TypeError, ValueError), и вы хотите указать комментарий к ошибке как мы видели ранее, чтобы пользователь сделал все как надо. Таких примеров очень много и вы понемногу станете с ними сталкиваться.

Чтобы бросить исключение необходимо воспользоваться командой `raise`:

In [102]:
x = int(input())
if x < 0:
    raise ValueError('Введенное значение должно быть неотрицательным!')

-2


ValueError: Введенное значение должно быть неотрицательным!

###  Особенная команда `assert`  

Эта команда вызывает ошибку `AssertionError`, если справа от `assert` стоит False-утверждение. По сути это еще один способ вызова исключения в более удобной форме, который можно использовать для каких-то простых проверок на True/False. 
```python
    assert логическое_выражение, "комментарий"
```

In [107]:
x = int(input())
assert x > 0, 'Введенное значение должно быть неотрицательным!'

0


AssertionError: Введенное значение должно быть неотрицательным!

###  Как ловить ошибку? `try...except`  

Хорошо, мы знаем какие ошибки бывают, как их вызывать самим в своем коде, знаем, что они могут возникнуть "встроенным" образом (например, если в коде баг). Теперь изучим как нам использовать исключения, чтобы в зависимости от их появления, мы могли подстроить код, чтобы он все равно успешно выполнился (или завершился понятным нам образом).

Полная форма конструкции обработки исключения выглядит таким образом:

```python
    try:
        <блок кода, в котором может возникнуть исключение>
    except <название исключения> [as e]:
        <блок обработки исключения>
    else:
        <блок кода, исполненный в случае, когда не возникает исключения>
    finally:
        <блок кода, гарантированно исполненный последним (т.е. всегда исполняется)>
```

Однако, чаще достаточно конструкции `try...except`. Не просто так эти две команды указаны вместе - они неразделимы! Нельзя написать только `try` или только `except`. Пример:

In [110]:
try:
    assert False, 'error'
except:
    pass

error


In [116]:
try:
    assert False, 'error'
except (ValueError, NameError): # Exception # AssertionError - попробуйте поменять
    print(1)
    pass

print(0)

AssertionError: error

In [117]:
try:
    assert False, 'error'
except Exception:
    print('Error occured')

Error occured


In [130]:
try:
    assert False, 'error'
except Exception as err:
    print(err)

error


In [104]:
try:
    assert False, 'error'
except:
    print('Something went wrong... Traceback:')
    raise

Something went wrong... Traceback:


AssertionError: error

**Usage case**

In [133]:
def div():
    try:
        x = float(input())
        y = float(input())
        answer = x ** y
    except ValueError:
        print('Неверное введенное значение для перевода в целое число!')
        return div()
    except ZeroDivisionError:
        print('Возникло деление на ноль!')
        answer = 0
    except:
        print('Ой! Ошибка которую я не ожидал от тебя! Вот она:')
        raise
    return answer

div() # можно заменить на while True: break конструкцию, вместо рекурсивной функции

o
Неверное введенное значение для перевода в целое число!
i
Неверное введенное значение для перевода в целое число!
p
Неверное введенное значение для перевода в целое число!
e
Неверное введенное значение для перевода в целое число!
d
Неверное введенное значение для перевода в целое число!
0
1


0.0

Однако, если мы все же используем `finally`, то надо быть аккуратным в следующем:

In [134]:
def one_or_zero():
    try:
        print(1)
        return 1
    except:
        return 10
    finally:
        return 0
    
one_or_zero()

1


0

##   Чтение и сохранение файлов  

In [137]:
f = open('input.txt', 'w')
f.close()

In [149]:
with open('input.txt', 'w') as file:
    file.write('hello\n')
    file.write('hey')
    file.writelines(('1\n', '2\n', '3'))

In [145]:
with open('input.txt', 'r') as file:
    print(file.name)
#     a = file.read(2)
#     b = file.read()
#     a = file.readlines()
    a = file.readline()
    b = file.readline()
    print(a)
    print(b)
    print(a)
a

input.txt
1

2

1



'1\n'

##   Модульное оформление  

Что такое модули? Простыми словами это папка с файлами, внутри которых написан код. Между собой файлы могут использовать функции, константные переменные. Есть много встроенных библиотек, а остальные можно установить с помощью `pip`.

In [155]:
import os
os.path.join('mnt', 'c', 'svdcvt')

'mnt/c/svdcvt'

In [ ]:
import time
import random
s = time.time()
random.random()
print(time.time() - s)

In [160]:
%timeit random.random()
%timeit random.randrange(0, 10)

390 ns ± 69.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
4.01 µs ± 829 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [1]:
import math as m
m.e

NameError: name 'math' is not defined

In [3]:
from math import e as exponent, pi as pinumber
exponent, pinumber

2.718281828459045

Мы можем написать модуль сами. Пусть в папке `mymodule` лежит два файла - `utils.py` `run.py`, и эта папка находится там же, где сейчас лежит этот ноутбук (или там же где мы находимся в командной строке).

In [4]:
from mymodule import utils 

utils.hello()
utils.constant 

Hello world!!!


10042

In [1]:
!python3 mymodule/run.py

Hello world!!!
10042
3


##   Тестирование  

###   Вступление  

Вы закончили курс по Питону и решили сделать крутой стартап - сервис по решению уравнений. Сначала вы набросали версию с минимальным функционалом, т.е. сервис умеет решать только линейные уравнения:


```python
def solve(a=1, b=0, c=0):
    '''
    ax + b = c
    '''
    return (c - b) / a

```

Как только код был написан, вы решили убедиться что ваша программа работает


```python
print(solve(a=2, b=1))
print(solve(b=2, c=3))
```

    >>>-0.5 
    >>>1.0

Отлично! Похоже что всё хорошо и можно релизить сервис на людей, что вы и делаете. Всё идет отлично, люди пользуются сервисом и вполне счастливы, только вот многие хотят, чтобы вы стали решать еще и квадратные уравнения. Вы делаете правку в своем коде:


```python
def solve_2(a=1, b=0, c=0, d=0):
    '''
    ax^2 + bx + c = d 
    '''
    D = b ** 2 - 4 * a * (c - d)  
    Ds = abs(D) ** 0.5
    return (-b + Ds)/(2 * a), (-b - Ds)/(2 * a)
```

Как и в прошлый раз вы тестируете новую функциональность:


```python
print(solve_2(a=1, b=2, c=1))
```

    >>> (-1.0, -1.0) 

Ура, работает! Вы выкатываете новую версию сервиса и ждете восторженных отзывов. Вместо этого число пользователей начинает стремительно падать. Вы начинаете искать причину и выясняете, что после обновления ваш сервис перестал решать линейные уравнения. Вы проверяете в консоли:


```python
print(solve_2(b=2, c=1))
```
    >>> (-1.0, -1.0)
    
Действительно не работает! Вы внимательно смотрите на свой код и обнаруживаете баг. 

Сделав правки, вы проверяете работоспособность:


```python
def solve_3(a=0, b=0, c=0, d=0):
    '''
    ax^2 + bx + c = d 
    '''
    if a != 0:
        D = b ** 2 - 4 * a * (c - d)  
        Ds = abs(D) ** 0.5 
        return (-b + Ds) / (2 * a), (-b - Ds) / (2 * a)
    elif b != 0:
        return (d - c) / b
    else:
        raise ValueError('Invalid equation')

print(solve_3(a=1, b=2, c=1))
print(solve_3(a=0, b=2, c=1))
```

    >>>(-1.0, -1.0) 
    >>>-0.5 


Всё работает! Чтобы такое не повторялось, вы записываете, что после правок проверять надо как линейные, так и нелинейные уравнения. Вы выкатываете новый релиз сервиса, но пользователи уже ушли к конкуренту.

###   Автоматизируем тестирование  

Вместо того, чтобы кажды раз проверять в консоли работает код или нет, можно написать функцию вида:


```python
def check_solve(solve_fn):
    assert solve_fn(a=1, b=2, c=3) == 1.0
    assert solve_fn(a=2, b=1) == -0.5
    print('Всё работает!')
```

История нашего сервиса была бы немного другой. Проверяем первую итерацию:


```python
check_solution(solve)
```

    Всё работает!

Катим, видим что пользователи хотят решения квадратных уравнений, делаем правки, обновляем наш тест:


```python
def check_solve(solve_fn):
    assert solve_fn(a=0, b=1, c=2, d=3) == 1.0
    assert solve_fn(a=0, b=2, c=1) == -0.5
    
    assert solve_fn(a=1, b=2, c=1) == (-1.0, -1.0)
    print('Всё работает!')
```

```python
check_solve(solve_2)
```
    ---------------------------------------------------------------------------
    AssertionError                            Traceback (most recent call last)
    <ipython-input-123-969194e3e673> in <module>
    ----> 1 check_solve(solve_2)

    <ipython-input-123-969194e3e673> in check_solve(solve_fn)
          1 def check_solve(solve_fn):
    ----> 2     assert solve_fn(a=0, b=1, c=2, d=3) == 1.0
          3     assert solve_fn(a=0, b=2, c=1) == -0.5
          4 

    AssertionError: 

Ага, что-то не так! Разбираемся, находим баг, правим:


```python
check_solve(solve_3)
```

    Всё работает!


Выкатываем, пользователи счастливы.

Такой подход называется **автоматизированным тестированием**. Для автоматизированного тестирования написана куча фреймворков на разных языках. Короткий список для python:

* [unittest](https://docs.python.org/3/library/unittest.html)
* [nose2](https://docs.nose2.io/en/latest/)
* [pytest](https://docs.pytest.org/en/latest/)

В рамках лекции мы остановимся на `pytest`, т.к. он найболее прост, лаконичен и поддерживается лучше других фреймворков.

###  Pytest   [ipytest](https://github.com/chmp/ipytest)

In [ ]:
!pip3 install pytest ipytest logging

In [4]:
# Настраиваем ноутбук
import pytest
import ipytest
ipytest.config(rewrite_asserts=True, magics=True)
__file__ = 'lecture09_adv1_slides.ipynb'

In [7]:
def solve(a=1, b=0, c=0):
    '''
    ax + b = c
    '''
    return (c - b) / a

def solve_2(a=1, b=0, c=0, d=0):
    '''
    ax^2 + bx + c = d 
    '''
    D = b ** 2 - 4 * a * (c - d)  
    Ds = abs(D) ** 0.5
    return (-b + Ds)/(2 * a), (-b - Ds)/(2 * a)

def solve_3(a=0, b=0, c=0, d=0):
    '''
    ax^2 + bx + c = d 
    '''
    if a != 0:
        D = b ** 2 - 4 * a * (c - d)  
        Ds = abs(D) ** 0.5 
        return (-b + Ds) / (2 * a), (-b - Ds) / (2 * a)
    elif b != 0:
        return (d - c) / b
    else:
        raise ValueError('Invalid equation')

###  Основы   

Тесты для pytest выглядят как функции, вызывающие `assert` внутри. Чтобы фреймворк подхватил функцию, её имя должно начинаться на `test`. Напишем один из тестов для нашего сервиса по решение уравнений:

In [11]:
%%run_pytest[clean] -q

def test_linear_equation():
    assert solve(2, 1) == -0.5

.                                                                                                                [100%]
1 passed in 0.01s


`Pytest` перехватывает вызов оператора `assert` и собирает подробную информацию о причине падения. Запустим наш тест на падающей версии сервиса:

In [13]:
%%run_pytest[clean] -q

def test_positive_smile_removed():
    assert solve_2(b=2, c=1) == -0.5, 'Функция должна решать линейные уравнения'

F                                                                                                                [100%]
======================================================= FAILURES =======================================================
_____________________________________________ test_positive_smile_removed ______________________________________________

    def test_positive_smile_removed():
>       assert solve_2(b=2, c=1) == -0.5, 'Функция должна решать линейные уравнения'
E       AssertionError: Функция должна решать линейные уравнения
E       assert (-1.0, -1.0) == -0.5
E        +  where (-1.0, -1.0) = solve_2(b=2, c=1)

<ipython-input-13-9b0523c6bc7e>:2: AssertionError
=============================================== short test summary info ================================================
FAILED lecture09_adv1_slides.py::test_positive_smile_removed - AssertionError: Функция должна решать линейные уравнения
1 failed in 0.02s


Видим, какой тест падает, а так же разницу между ожидаемым результатом и реальной работой нашего кода.

`Pytest` поддерживает работу со встроенными типами данных. Несколько примеров:

In [14]:
%%run_pytest[clean] -q

def test_list():
    assert [1, 2, 3] == [2, 3]

def test_set():
    assert {1, 2, 3} == {2, 3}
    
def test_dict():
    assert {"a": 1} == {"a": 1, "b": 2}

FFF                                                                                                              [100%]
======================================================= FAILURES =======================================================
______________________________________________________ test_list _______________________________________________________

    def test_list():
>       assert [1, 2, 3] == [2, 3]
E       assert [1, 2, 3] == [2, 3]
E         At index 0 diff: 1 != 2
E         Left contains one more item: 3
E         Full diff:
E         - [2, 3]
E         + [1, 2, 3]
E         ?  +++

<ipython-input-14-7e37b465bc2b>:2: AssertionError
_______________________________________________________ test_set _______________________________________________________

    def test_set():
>       assert {1, 2, 3} == {2, 3}
E       assert {1, 2, 3} == {2, 3}
E         Extra items in the left set:
E         1
E         Full diff:
E         - {2, 3}
E         + {1, 2, 3}
E         ?  +++

###   Тестирование исключений   

Вспомним как у нас функция валидирует входные коэффициенты на валидность уравнения:

Тест может выглядеть так:

In [15]:
%%run_pytest[clean] -q

def test_error_on_invalid_equation():
    with pytest.raises(ValueError):
        solve_3(0, 0, 0, 0)

.                                                                                                                [100%]
1 passed in 0.02s


###   Сравнение вещественных чисел  

Из-за ошибок округления `float` трудно сравнивать через `==`

In [20]:
%%run_pytest[clean] -q

def test_float():
    assert 0.1 + 0.2 == 0.3

F                                                                                                                [100%]
======================================================= FAILURES =======================================================
______________________________________________________ test_float ______________________________________________________

    def test_float():
>       assert 0.1 + 0.2 == 0.3
E       assert (0.1 + 0.2) == 0.3

<ipython-input-20-d4fdc5effe72>:2: AssertionError
=============================================== short test summary info ================================================
FAILED lecture09_adv1_slides.py::test_float - assert (0.1 + 0.2) == 0.3
1 failed in 0.01s


Исправить ситуацию поможет `pytest.approx`

In [17]:
%%run_pytest[clean] -q

def test_float():
    assert 0.1 + 0.2 == pytest.approx(0.3)

.                                                                                                                [100%]
1 passed in 0.01s


`pytest.approx` так же работает и с множественными значениями:

In [21]:
%%run_pytest[clean] -q

def test_float():
    assert [0.1 + 0.2, 0.5] == pytest.approx([0.3, 0.5])

.                                                                                                                [100%]
1 passed in 0.01s


###   Параметризованные тесты   

Вспомним тест для нашего сервиса:

In [25]:
%%run_pytest[clean] -q

def test_equation():
    assert solve_3(a=0, b=1, c=2, d=3) == 1.0
    assert solve_3(a=0, b=2, c=1) == -0.5
    assert solve_3(a=1, b=2, c=1) == (-1.0, -1.0)

.                                                                                                                [100%]
1 passed in 0.01s


Хочется добавить еще кейсов. Сделать это можно так:

In [26]:
%%run_pytest[clean] -q

def test_equation():
    assert solve_3(a=0, b=1, c=2, d=3) == 1.0
    assert solve_3(a=0, b=2, c=1) == -0.5
    assert solve_3(a=0, b=-2, c=1) == 0.5
    assert solve_3(a=1, b=2, c=1) == (-1.0, -1.0)
    assert solve_3(a=1, b=-2, c=1) == (1.0, 1.0)

.                                                                                                                [100%]
1 passed in 0.01s


У такого подхода есть как минимум два недостатка:
1. Нужно каждый раз писать один и тот же код для вызова функции (boilerplate)
2. Тест упадет на первом плохом assert'e, и мы не узнаем какие у нас еще есть баги пока не починим первый

К счастью, `pytest` предоставляет нам удобный инструментарий для параметризации тестов:

In [51]:
%%run_pytest[clean] -q

test_cases = [((0, 1, 2, 3), 1.0), 
              ((0, 2, 1), -0.5), 
              ((0, -2, 1), 0.5),
              ((1, 2, 1), (-1.0, -1.0)),
              ((1, -2, 1), (1.0, 1.0))
             ]
test_names = ['test_lin_a', 'test_lin_b', 'test_lin_c', 'test_quad_a', 'test_quad_b'] # опционально

@pytest.mark.parametrize("input_message, expected_message", test_cases, ids=test_names)
def test_equation(input_message, expected_message):
    assert solve_3(*input_message) == expected_message # не забудьте про звездочку

.....                                                                                                            [100%]
5 passed in 0.03s


###   Test discovery  

Для запуска тестов нужно вызвать `pytest` на папку проекта. Тогда `pytest`:
1. Рекурсивно обойдет все подпапки
2. Найдет все файлы, с именами вида `test_*.py` или `*_test.py`
3. В этих файлах запустит как тест все функции, начинающиеся с `test`.

Соответственно, здесь мы воспользуемся модульным оформлением нашего кода, функции которого мы будем импортировать в файлах тестов.
    
Куда лучше сложить тесты в вашем проекте можно почитать [здесь](https://blog.ionelmc.ro/2014/05/25/python-packaging/#the-structure). Таким же похожим образом можно делать тесты для проекта в гитхабе.

###   Best practices   

**1: Проверяйте, что фейлится так как вы ожидаете**

Вы решили написать новый тест, и чтобы тест проверял побольше, вы решили написать генератор тесткейсов. НО! Если в самом тесте есть баг (например, генерируем пустое множество тесткейсов и на самом деле вообще ничего не проверяем), то тесты будут проходиться и всё будет казаться будто бы хорошо. 

Поэтому:
1. Если вы пишете тест на существующий код, внесите в него правки, из-за которых тест упадет ожидаемым образом. После чего исправьте код так, чтобы тест проходил.
2. Если вы нашли баг в коде, напишите падающий тест до исправления этого бага.
3. Если пишите новый код, напишите тесты заранее.

**2: Старайтесь покрыть тестами все ветви исполнения вашего кода**

Отношение покрытых тестами строчек к общему числу строчек кода называется **code coverage**.
Есть куча инструментов для ее измерения, например [coverage.py](https://coverage.readthedocs.io/en/v4.5.x/). Многие opensoruce проекты вешают плашку с текущим coverage у себя в git. Рекомендуется поддерживать сoverage близким к 100%.

**3: Среднее число логических assert в тестах должно стремиться к 1**

Если у вас падает какой-то тест, то вы должны иметь возможность как можно быстрее понять какая именно часть вашего кода не работает. Если в тесте много assert'ов, вам скорее придется найти код теста и разбираться что именно проверялось в каждом из них. Вместо такого:

In [57]:
%%run_pytest[clean] -q

def test_equation():
    assert solve_3(a=0, b=1, c=2, d=3) == 1.0
    assert solve_3(a=0, b=2, c=1) == -0.5
    assert solve_3(a=1, b=2, c=1) == (-1.0, -1.0)

.                                                                                                                [100%]
1 passed in 0.01s


Лучше сделать так:

In [58]:
%%run_pytest[clean] -q

def test_linear():
    assert solve_3(a=0, b=1, c=2, d=3) == 1.0
    assert solve_3(a=0, b=2, c=1) == -0.5
def test_quadr_same():
    assert solve_3(a=1, b=2, c=1) == (-1.0, -1.0)
def test_qadr_different():
    assert solve_3(1, 2, 3) == pytest.approx((0.414, -2.414), rel=1e-3)

...                                                                                                              [100%]
3 passed in 0.02s


**4: именуйте тесты понятно**

Вы не будете вызывать ваши функции с тестами где-то в коде, значит экономить на длинне названия бессмысленно. С другой стороны хочется понимать из названия теста как что подается на вход и как должен работать при этом код.

**5: Проверяйте граничные условия и диапазоны**


###   Профиты  

1. Не забываем про то, какую функциональность имеет наш код
2. Можем вносить правки в код не боясь что-то упустить
3. Можем понять, как работать с нашим кодом исходя из тестов

**Книги**
1. [Test Driven Development: By Example 1st Edition](https://www.eecs.yorku.ca/course_archive/2003-04/W/3311/sectionM/case_studies/money/KentBeck_TDD_byexample.pdf)
2. [On Growing Object Oriented Software, Guided by Tests](https://www.amazon.com/Growing-Object-Oriented-Software-Guided-Tests/dp/0321503627)
3. [The art of unit testing](http://artofunittesting.com/definition-of-a-unit-test)

**Статьи**
1. Про то, куда сложить тесты - [Packaging a python library](https://blog.ionelmc.ro/2014/05/25/python-packaging/#the-structure)
2. Про именование тестов - [Youre naming tests wrong](https://enterprisecraftsmanship.com/posts/you-naming-tests-wrong/)

##   Логирование  

In [12]:
import sys
import logging

logger = logging.getLogger('my_logger') # Создаем объект "логер", он записывает логи

stdout_handler = logging.StreamHandler(stream=sys.stdout) # Говорим куда логировать (хэндлер), он хранит логи
stdout_handler.setLevel(logging.INFO) # Выставляем уровень логирования 
logger.addHandler(stdout_handler) # Добавляем к логеру "хэндлер"

In [3]:
logger.warning('Watch out!') # пишем лог

Watch out!


###   Логгирование и его уровни    

Логгировать можно при помощи следующих методов:
1. `logger.debug()`
2. `logger.info()`
3. `logger.warning()`
4. `logger.error()`
5. `logger.critical()`
6. `logger.exception()`

С самого маленького до самого большого

|Уровень | Когда используется|
|:------ |:------------------|
|`DEBUG`|Для диагностической информации|
|`INFO`|Для подтверждения того, что всё работает как запланировано|
|`WARNING`|Когда нужно предупредить что вскоре возможна поломка или программа используется не совсем так как нужно|
|`ERROR`|Для логгирования сепъезных ошибок, из-за которых программа теряет часть функциональности|
|`CRITICAL`|Для логгирования ошибок, после которых программа не может продолжать работу|

Стандартный уровень логгирования - `WARNING`

In [4]:
logger.info('Will not be printed')
logger.warning('Will be printed')

Will be printed


Поменяем уровень логгирования:

In [5]:
logger.setLevel(logging.INFO)
logger.info('Will not be printed')
logger.warning('Will be printed')
logger.setLevel(logging.WARNING)
logger.log(logging.WARNING, 'Will be printed')

Will not be printed
Will be printed
Will be printed


`logging.exception` немножко особенный - он добавляет информацию о последнем исключении и traceback

In [6]:
try:
    1 / 0
except:
    logger.exception("Cought error:")

Cought error:
Traceback (most recent call last):
  File "<ipython-input-6-7090cd0566ef>", line 2, in <module>
    1 / 0
ZeroDivisionError: division by zero


###   Логгирование в файл и хэндлеры   

Научимся логгировать в файл

In [7]:
file = logging.FileHandler('debug.log') # Создаем файл, где будут храниться логи
file.setLevel(logging.DEBUG)  # Выставляем уровень сообщений, которые будут логгироваться в файл.
logger.addHandler(file) # Сообщаем логгеру куда можно писать логи

In [8]:
logger.setLevel(logging.DEBUG) # Выставляем уровень логирования
logger.debug("Debug message")  # Не попадет в stdout, зато попадет в файл
logger.debug("HI!")
logger.debug("Debug message!")

# теперь откройте ваш файл и посмотрите что там

Другие полезные хендлеры из библиотеки `logging`:

* `StreamHandler` - используется для логгирования в `stderr` и `stdout`
* `RotatingFileHandler` - Работает вкак файл хендлер, но при этом если файл, в который пишет логгер, достигнет определенного размера, начнет писать в новый файл. Старый файл либо удалит, либо оставит как бекап. Число бэкапов настраивается. 
* `TimedRotatingFileHandler` - Работает как логгер выше, но файлы делятся не по размеру, а по времени записей
* `NullHandler` - Используется чтобы заглушить какой-нибудь логгер

###   Фильтрация    

На логгеры и хендлеры можно вешать дополнительные фильтры

In [9]:
logger.warning("Not important warning")

class OnlyImportantFilter(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith('Not important')

logger.addFilter(OnlyImportantFilter())
logger.warning("Not important warning")

Not important warning


###   Форматирование   

Для хендлера можно выставить формат при помощи метода `setFormatter`

In [10]:
format_example_logger = logging.getLogger('format_example')
format_example_logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', '%m-%d %H:%M')

ch.setFormatter(formatter)
format_example_logger.addHandler(ch)

In [11]:
format_example_logger.debug('debug message')
format_example_logger.info('info message')
format_example_logger.warning('warn message')
format_example_logger.error('error message')
format_example_logger.critical('critical message')

11-16 17:22 - format_example - DEBUG - debug message
11-16 17:22 - format_example - INFO - info message
11-16 17:22 - format_example - WARNING - warn message
11-16 17:22 - format_example - ERROR - error message
11-16 17:22 - format_example - CRITICAL - critical message


`Formatter` принимает три аргумента: `fmt`, `datefmt`, `style`.

`fmt` - это шаблон записи. Если в `style` стоит %, форматирование произойдет при помощи %. Если `style` равен {, форматирование будет произведено через `.format()`. В `fmt` можно добавить любые атрибуты класса [LogRecord](https://docs.python.org/3/library/logging.html#logrecord-attributes)

`datefmt` - шаблон для форматирования дат, по дефолту `%Y-%m-%d %H:%M:%S`

##   Argparse  

Представим, что мы запускаем программу из командной строки, т.е. используем command-line interface (CLI). Супер подробно можно почитать [тут](https://docs.python.org/3/howto/argparse.html) и [тут](https://docs.python.org/3/library/argparse.html#module-argparse). Библиотека `argparse` позволяет нам получать аргументы из командной строки. Например, `example.py`:

```python
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("square", type=int,
                        help="display a square of a given number")
    parser.add_argument("-v", "--verbose", action="store_true",
                        help="increase output verbosity")
    args = parser.parse_args()
    answer = args.square ** 2
    if args.verbose:
        print("the square of {} equals {}".format(args.square, answer))
    else:
        print(answer)
```

In [65]:
!python3 example.py 

usage: example.py [-h] [-v] square
example.py: error: the following arguments are required: square


In [33]:
!python3 example2.py 10 2 -vv

10 to the power 2 equals 100


In [45]:
!python3 example3.py x -a 10 -c 3 -vv

The equation: (10.0)x^2 + (0)x + (3.0) = 0


In [62]:
!python3 example4.py -c 1 2 3 4 x -vv

Namespace(char='x', coef=[1, 2, 3, 4], verbosity=2)
The equation: (1)x^2 + (2)x + (3) = 4
